## 二、 使用 LoRA 进行微调（cosmosqa数据集）

命令行一键运行

设备：M40-24G

基座模型：`MiniCPM`

使用数据：`cosmosQA`

由于显存限制，训练时的`per_device_train_batch_size`取1，本实验在模型、数据、其他参数都相同的情况下，分别让`gradient_accumulation_steps`取2、4、8、16，这等效于`Total train batch size`分别为2、4、8、16。并对比分析测试集的准确率。

### 修改lora_finetune_cosmosQA.sh的内容为如下：

```bash
formatted_time=$(date +"%Y%m%d%H%M%S")
echo $formatted_time

#4090添加这两行代码
export NCCL_P2P_DISABLE=1
export NCCL_IB_DISABLE=1 

deepspeed --include localhost:0 --master_port 19888 finetune.py \
    --model_name_or_path /hy-tmp/miniCPM-bf16 \
    --output_dir output/LoRA/$formatted_time/ \
    --train_data_path /hy-tmp/data_MiniCPM/train.json \
    --eval_data_path /hy-tmp/data_MiniCPM/valid.json \
    --learning_rate 5e-5 \
    --per_device_train_batch_size 1 \
    --per_device_eval_batch_size 4 \
    --model_max_length 1024 \
    --use_lora \
    --gradient_accumulation_steps 2 \
    --warmup_steps 100 \
    --max_steps 1000 \
    --weight_decay 0.01 \
    --evaluation_strategy steps \
    --eval_steps 500 \
    --save_strategy steps \
    --save_steps 500 \
    --seed 42 \
    --log_level info --logging_strategy steps --logging_steps 10 \
    --deepspeed configs/ds_config_zero2_offload.json
```


In [2]:
"""
微调模型1
--per_device_train_batch_size 1
--gradient_accumulation_steps 2
"""

!bash lora_finetune_cosmosQA.sh

20240725165943
[2024-07-25 16:59:45,251] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  NVIDIA Inference is only supported on Ampere and newer architectures
 [WARNING]  NVIDIA Inference is only supported on Pascal and newer architectures
 [WARNING]  NVIDIA Inference is only supported on Pascal and newer architectures
 [WARNING]  NVIDIA Inference is only supported on Pascal and newer architectures
 [WARNING]  NVIDIA Inference is only supported on Pascal and newer architectures
 [WARNING]  NVIDIA Inference is only supported on Pascal and newer architectures
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.3
 [WARNING]  using untested triton version (2.3.1), only 1.0.0 is known to be compatible
[2024-07-25 16:59:47,560] [WARNING] [runner.py:202:fetch_hostfile] Unable to find hostfile, will proceed with traini

In [1]:
"""
微调模型2
--per_device_train_batch_size 1
--gradient_accumulation_steps 4
"""

!bash lora_finetune_cosmosQA.sh

20240728001511
[2024-07-28 00:15:13,108] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  NVIDIA Inference is only supported on Ampere and newer architectures
 [WARNING]  NVIDIA Inference is only supported on Pascal and newer architectures
 [WARNING]  NVIDIA Inference is only supported on Pascal and newer architectures
 [WARNING]  NVIDIA Inference is only supported on Pascal and newer architectures
 [WARNING]  NVIDIA Inference is only supported on Pascal and newer architectures
 [WARNING]  NVIDIA Inference is only supported on Pascal and newer architectures
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.3
 [WARNING]  using untested triton version (2.3.1), only 1.0.0 is known to be compatible
[2024-07-28 00:15:15,507] [WARNING] [runner.py:202:fetch_hostfile] Unable to find hostfile, will proceed with traini

In [2]:
"""
微调模型3
--per_device_train_batch_size 1
--gradient_accumulation_steps 8
"""

!bash lora_finetune_cosmosQA.sh

20240728042026
[2024-07-28 04:20:28,607] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  NVIDIA Inference is only supported on Ampere and newer architectures
 [WARNING]  NVIDIA Inference is only supported on Pascal and newer architectures
 [WARNING]  NVIDIA Inference is only supported on Pascal and newer architectures
 [WARNING]  NVIDIA Inference is only supported on Pascal and newer architectures
 [WARNING]  NVIDIA Inference is only supported on Pascal and newer architectures
 [WARNING]  NVIDIA Inference is only supported on Pascal and newer architectures
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.3
 [WARNING]  using untested triton version (2.3.1), only 1.0.0 is known to be compatible
[2024-07-28 04:20:30,944] [WARNING] [runner.py:202:fetch_hostfile] Unable to find hostfile, will proceed with traini

In [3]:
"""
微调模型4
--per_device_train_batch_size 1
--gradient_accumulation_steps 16
"""

!bash lora_finetune_cosmosQA.sh

20240728122545
[2024-07-28 12:25:47,702] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  NVIDIA Inference is only supported on Ampere and newer architectures
 [WARNING]  NVIDIA Inference is only supported on Pascal and newer architectures
 [WARNING]  NVIDIA Inference is only supported on Pascal and newer architectures
 [WARNING]  NVIDIA Inference is only supported on Pascal and newer architectures
 [WARNING]  NVIDIA Inference is only supported on Pascal and newer architectures
 [WARNING]  NVIDIA Inference is only supported on Pascal and newer architectures
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.3
 [WARNING]  using untested triton version (2.3.1), only 1.0.0 is known to be compatible
[2024-07-28 12:25:50,042] [WARNING] [runner.py:202:fetch_hostfile] Unable to find hostfile, will proceed with traini

## 推理验证

In [1]:
import torch
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer

/usr/local/lib/python3.11/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
"""
加载微调模型1
"""

path = "/hy-tmp/MiniCPM/finetune/output/LoRA/20240725165943/checkpoint-1000"
tokenizer = AutoTokenizer.from_pretrained(path)
model = AutoModelForCausalLM.from_pretrained(
    path, torch_dtype=torch.float16, device_map="cuda", trust_remote_code=True
)

设计一条query看看结果

In [3]:
query = "<用户>You are a reading comprehension expert and below you need to answer multiple choice questions based on the text. Note: Only the number of the option needs to be answered and your answer is only one number.\n" \
         "Here are the questions you have to answer: \n" \
         "Text: HGH and steroid use is rampant in track , and , just like in most American professional sports , any incredible individual achievement is questioned immediately , which is a sad state of affairs . The fact that we , as a human population have to assume that someone is using steroids because of an outstanding feat is horrible . What about the people who actually train their hearts out to achieve the same success that the users are trying to achieve ? It is n't fair to the men and women that dedicate their lives to achieving glory the right way , hard work and determination .\n" \
         "Question: Why is HGH and steroid use rampant in track ?\n" \
         "Option 0: Because we have to assume that someone is using steroids because of an outstanding feat .\n" \
         "Option 1: None of the above choices .\n" \
         "Option 2: Because any incredible individual achievement is questioned immediately .\n" \
         "Option 3: Because it 's an American professional sport .\n" \
         
res, history = model.chat(tokenizer, query=query, max_length=1024, top_p=0.5)
res, history

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.


('2',
 [{'role': 'user',
   'content': "<用户>You are a reading comprehension expert and below you need to answer multiple choice questions based on the text. Note: Only the number of the option needs to be answered and your answer is only one number.\nHere are the questions you have to answer: \nText: HGH and steroid use is rampant in track , and , just like in most American professional sports , any incredible individual achievement is questioned immediately , which is a sad state of affairs . The fact that we , as a human population have to assume that someone is using steroids because of an outstanding feat is horrible . What about the people who actually train their hearts out to achieve the same success that the users are trying to achieve ? It is n't fair to the men and women that dedicate their lives to achieving glory the right way , hard work and determination .\nQuestion: Why is HGH and steroid use rampant in track ?\nOption 0: Because we have to assume that someone is using s

In [4]:
res

'2'

In [5]:
# 看一下没有微调过的模型是什么结果

MiniCPM_bf16_path = "/hy-tmp/miniCPM-bf16"
MiniCPM_bf16_tokenizer = AutoTokenizer.from_pretrained(MiniCPM_bf16_path)
MiniCPM_bf16_model = AutoModelForCausalLM.from_pretrained(
    MiniCPM_bf16_path, torch_dtype=torch.float16, device_map="cuda", trust_remote_code=True
)
         
MiniCPM_bf16_res, MiniCPM_bf16_history = MiniCPM_bf16_model.chat(MiniCPM_bf16_tokenizer, query=query, max_length=1024, top_p=0.5)
MiniCPM_bf16_res

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'Option 2: Because any incredible individual achievement is questioned immediately.'

使用测试集跑一下

In [ ]:
import json
import csv
import re
import os

input_file_path = '/hy-tmp/cosmosqa-master/data/test.jsonl'
output_file_path = os.path.join(path, 'predictions_cosmosqa.csv')

# 逐行读取 JSONL 文件
with open(input_file_path, 'r') as file, open(output_file_path, 'w', newline='') as csvfile:
    # 初始化列表来存储 id 和 label
    data = []
    writer = csv.writer(csvfile)
    writer.writerow(['id', 'label'])    
    for line in file:
        record = json.loads(line)
        # 提取 id 和 label 信息
        id = record['id']
        input = f"<用户>You are a reading comprehension expert and below you need to answer multiple choice questions based on the text. Note: Only the options need to be answered, no additional explanations are needed.\n" \
         f"Here are the questions you have to answer: \n" \
         f"Text: {record['context']}\n" \
         f"Question: {record['question']}\n" \
         f"Option 0: {record['answer0']}\n" \
         f"Option 1: {record['answer1']}\n" \
         f"Option 2: {record['answer2']}\n" \
         f"Option 3: {record['answer3']}\n"
        res, history = model.chat(tokenizer, query=input, max_length=1024, top_p=0.5)
        label = re.sub(r'\D', '', res)
        writer.writerow([id, label])

In [14]:
import csv
import os

# 输入 CSV 文件路径
csv_file_path = output_file_path
# 输出 LST 文件路径
lst_file_path = os.path.join(path, 'predictions.lst')

# 打开 CSV 文件并读取内容
with open(csv_file_path, 'r') as csvfile:
    csv_reader = csv.reader(csvfile)
    
    # 跳过第一行
    next(csv_reader)
    
    # 打开 LST 文件以写入
    with open(lst_file_path, 'w') as lstfile:
        for row in csv_reader:
            # 写入 label (假设标签在第二列)
            label = row[1]
            lstfile.write(f"{label}\n")


In [4]:
"""
用没有微调过的模型，跑一下准确度
由于模型给出的回答是形如
'Option 2: Because any incredible individual achievement is questioned immediately.'的格式
所以加了提取选项的一步
预测文件名为predictions_cosmosqa_0.lst
"""

import torch
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer
import json
import re
import os

path = '/hy-tmp/MiniCPM/finetune/output/results'
input_file_path = '/hy-tmp/cosmosqa-master/data/test.jsonl'
output_file_path = os.path.join(path, 'predictions_cosmosqa_0.lst')

MiniCPM_bf16_path = "/hy-tmp/miniCPM-bf16"
MiniCPM_bf16_tokenizer = AutoTokenizer.from_pretrained(MiniCPM_bf16_path)
MiniCPM_bf16_model = AutoModelForCausalLM.from_pretrained(
    MiniCPM_bf16_path, torch_dtype=torch.float16, device_map="cuda", trust_remote_code=True
)

# 逐行读取 JSONL 文件
total_lines = sum(1 for line in open(input_file_path, 'r'))
with open(input_file_path, 'r') as file, open(output_file_path, 'w') as lstfile:
    for line in tqdm(file, total=total_lines, desc="Processing"):
        record = json.loads(line)
        input = f"<用户>You are a reading comprehension expert and below you need to answer multiple choice questions based on the text. Note: Only the number of the option needs to be answered and your answer is only one number.\n" \
         f"Here are the questions you have to answer: \n" \
         f"Text: {record['context']}\n" \
         f"Question: {record['question']}\n" \
         f"Option 0: {record['answer0']}\n" \
         f"Option 1: {record['answer1']}\n" \
         f"Option 2: {record['answer2']}\n" \
         f"Option 3: {record['answer3']}\n"
        res, history = MiniCPM_bf16_model.chat(MiniCPM_bf16_tokenizer, query=input, max_length=1024, top_p=0.5)
        label = res.split(":")[0]
        label = re.sub(r'\D', '', label)
        lstfile.write(f"{label}\n")

Processing:   0%|          | 0/6963 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
Processing: 100%|██████████| 6963/6963 [5:42:49<00:00,  2.95s/it]
